In [1]:
!rm -rf wiki_libs
!git clone https://github.com/tsonic/wiki_libs.git

Cloning into 'wiki_libs'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 84 (delta 43), reused 60 (delta 25), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import json
from collections import defaultdict
import pandas as pd
from zipfile import ZipFile
import pickle
import gc
from gensim.models.phrases import Phrases, Phraser
import itertools
import importlib
import wiki_libs
from wiki_libs.preprocessing import *
from functools import partial
import torch.optim as optim
import traceback
importlib.reload(wiki_libs.preprocessing)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
# cudnn.benchmark = True # CUDA for PyTorch. Faster runtime if input size constant in iterations.  
!pip install ipdb > /dev/null
!pip install line_profiler > /dev/null
%load_ext line_profiler
import ipdb
import pdb

In [3]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


Configuration Class

In [ ]:
%%time
df_cp = read_category_links()

In [ ]:
%%time
#load ngram model

ngram_model = load_ngram_model(NGRAM_MODEL_PATH_PREFIX + "title_category_ngram_model.pickle")

In [ ]:
print(df_cp.columns)
print(df_cp.shape)

In [ ]:
df_cp['page_title'].isnull().mean()

In [ ]:
%%time
# tranform title, return list of list of ngrams
title_transformed, category_transformed = get_transformed_title_category(ngram_model)

In [ ]:
vocab_title = generate_vocab(title_transformed)

In [ ]:
vocab_category = generate_vocab(category_transformed)

In [ ]:
vocab_title_category = generate_vocab(title_transformed + category_transformed)

In [ ]:
# with open('gdrive/My Drive/Projects with Wei/wiki_data/ngram_model/vocab.json','w') as f:
#   json.dump({
#       'vocab_title':vocab_title,
#       'vocab_category':vocab_category,
#       'vocab_title_category':vocab_title_category,
#       }), f)

In [ ]:
with open('gdrive/My Drive/Projects with Wei/wiki_data/ngram_model/vocab.json','r') as f:
    vocab_json = json.load(f)
    vocab_title = vocab_json['vocab_title']
    vocab_category = vocab_json['vocab_category']
    vocab_title_category = 'vocab_title_category'
    word2ind_title_category = defaultdict(lambda:-1, {w:i for i, w in enumerate(vocab_title_category)})
    word2ind_title = defaultdict(lambda:-1, {w:i for i, w in enumerate(vocab_title)})
    word2ind_category = defaultdict(lambda:-1, {w:i for i, w in enumerate(vocab_category)})

In [ ]:
len(vocab_title)


In [ ]:
def ngram_to_idx(l, word2ind):
  return np.array([word2ind[ng] for ng in l])

In [ ]:
%%time
df_page_2_bow = (
    df_cp[['page_id','page_title']]
    .drop_duplicates()
    .fillna({'page_title':''})
    .assign(processed_title = lambda df: df['page_title'].apply(process_title))
    .assign(processed_title_ngram = lambda df: 
            transform_ngram(df['processed_title'].tolist(), ngram_model))
    .assign(processed_title_ngram_idx = lambda df: 
            df['processed_title_ngram'].apply(ngram_to_idx, word2ind=word2ind_title))
)

In [ ]:
%%time
df_category_2_bow = (
    df_cp[['page_category']]
    .drop_duplicates()
    .fillna({'page_category':''})
    .assign(processed_category = lambda df: df['page_category']
                                      .apply(process_title))
    .assign(processed_category_ngram = lambda df: transform_ngram(df['processed_category'].tolist(), ngram_model))
    .assign(processed_category_ngram_idx = 
                                     lambda df: df['processed_category_ngram'].apply(ngram_to_idx, word2ind=word2ind_category))
)

In [ ]:
%%time
df_lp = next(read_link_pairs_chunks(n_chunk=10))
df_lp.shape

In [ ]:
class Config():
    #TODO: fill in the dirs
    training_dir = \
    testing_dir = \

    #TODO: complete hyper-par defs
    EMD_DIM1      = 2 ** 3 #left tower 
    EMD_DIM2      = 2 ** 3 #right tower (= left for L2 cals)
    MAX_EPOCHS    = 100
    LEARNING_RATE = 1 ** (-5) # grid \in [-6:-3]
    NUM_EPOCHS     = 10

    # data parameters. to be used in the DataLoader caclass
    data_params = {'BATCH_SIZE': 2 ** 6, # grid \in [3:8]
                   'shuffle': True,
                   'num_workers': 6}


Helper_functions

In [ ]:
def show_plot(iteration, loss):
    plt.plot(iteration, loss)
    plt.show()

**Import** data

Reference: https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

In [4]:
class PageWordStats(object):
    def __init__(self, read_path,
                 ngram_model_file = "title_category_ngram_model.pickle",
                 output_path = "gdrive/My Drive/Projects with Wei/wiki_data/page_word_stats.json",
                 n_chunk = 10,
                 ):
        if read_path is not None:
            config = json.load(open(read_path, 'r'))
            self.word_frequency = config['word_frequency']
            self.word2id = config['word2id']
            self.id2word = config['id2word']
            self.page_frequency = {int(k):v for k, v in config['page_frequency'].items()}

            self.page2id = {int(k):v for k, v in config['page2id'].items()}
            self.id2page = config['id2page']
        else:
            # recomptue the target page stats

            gen = read_link_pairs_chunks(n_chunk = n_chunk)
            print('generating page id stats...')
            s_page = []
            for df_chunk in gen:
                val_counts = df_chunk['page_id_target'].append(df_chunk['page_id_source']).value_counts()
                s_page.append(val_counts)
            df_stats = (
                pd.concat(s_page)
                .rename_axis("page_id")  #rename index
                .to_frame('count')
                .groupby("page_id")
                    .sum()
                .reset_index()
            )
            self.page_frequency = {row.page_id: row.count 
                                   for row in df_stats.itertuples(index = False)}
            self.page2id = {p:i for i, p in enumerate(df_stats['page_id'])}
            self.id2page = df_stats['page_id'].tolist()

            # recompute the word stats
            print('generating word/ngram stats from title and categories...')

            ngram_model = load_ngram_model(NGRAM_MODEL_PATH_PREFIX + ngram_model_file)
            title_transformed, category_transformed = get_transformed_title_category(ngram_model)
            s_words = (
                pd.Series(itertools.chain(*(title_transformed + category_transformed)))
                .value_counts()
            )
            # word_frequency is a list, where ith element is the word frequency of word with id i.
            self.word_frequency = s_words.tolist()
            self.word2id = {w:i for i, w in enumerate(s_words.index)}
            self.id2word = s_words.index.tolist()
            json.dump({
                'word_frequency': self.word_frequency,
                'word2id': self.word2id,
                'id2word':self.id2word,
                'page_frequency': self.page_frequency,
                'page2id': self.page2id,
                'id2page': self.id2page,
                }, 
                open(output_path, 'w'))
        print('There are %d unique words/ngrams' % len(self.word2id))
        print('There are %d unique pages' % len(self.page2id))

In [5]:
# %%time
# page_word_stats = PageWordStats(read_path = None,n_chunk=10)

In [6]:
page_word_stats = PageWordStats(read_path = "gdrive/My Drive/Projects with Wei/wiki_data/page_word_stats.json")

There are 1748542 unique words/ngrams
There are 5330812 unique pages


In [7]:
NEGATIVE_TABLE_SIZE = 1e8
class WikiDataset(torch.utils.data.IterableDataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, file_list, compression, n_chunk, num_negs, page_word_stats, ns_exponent):
        'Initialization'
        #self.labels = labels
        # self.list_IDs = list_IDs
        # if isinstance(file_list, str):
        #     file_list = [file_list]
        self.file_list = file_list
        self.compression = compression
        self.n_chunk = n_chunk
        self.pos = 0

        self.chunk_iterator = None
        self.instance_dict = None


        self.negatives = []
        self.negpos = 0
        self.num_negs = num_negs
        self.page_word_stats = page_word_stats

        self.word_frequency = page_word_stats.word_frequency
        self.word2id = page_word_stats.word2id
        self.id2word = page_word_stats.id2word
        self.page2id = page_word_stats.page2id
        p,i = zip(*self.page2id.items())
        self.page2id_series_map = pd.Series(i, index = p, dtype = np.int64)
        self.id2page = page_word_stats.id2page
        self.page_frequency = page_word_stats.page_frequency
        self.initTableNegatives(ns_exponent=ns_exponent)

    # Iterable may not know the length of the stream before hand
    # def __len__(self):
    #     'Denotes the total number of samples'
    #     return len(self.list_IDs)

    # def __getitem__(self, index):
    #     'Generates one sample of data'
    #     # Select sample
    #     ID = self.list_IDs[index]

    #     # Load data and get label
    #     X = torch.load('data/' + ID + '.pt')
    #     y = self.labels[ID]

    #     return X, y
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.chunk_iterator is None:
            self.chunk_iterator = read_files_in_chunks(self.file_list, compression=self.compression, 
                                                       n_chunk = self.n_chunk, progress_bar = True)
            
        if self.instance_dict is None or self.pos >= len(self.instance_dict):
            #print('read_new_chunk', flush = True)
            df = next(self.chunk_iterator)
            # have to reset self.pos after pull next in above iterator, otherwise 
            # it does not invalidate current instance_dict after iterator is exhausted.
            self.pos = 0

            df = df.assign(
                    page_id_source = lambda df: df['page_id_source'].map(self.page2id_series_map),
                    page_id_target = lambda df: df['page_id_target'].map(self.page2id_series_map),
                )

            self.instance_dict = list(df.itertuples(index = False, name = None))
        ret = self.instance_dict[self.pos]
        self.pos += 1
        return ret

    def getNegatives(self, target, size):  # TODO check equality with target
        response = self.negatives[self.negpos:self.negpos + size]
        # reshuffle negative table if negpos > total neg table size.
        if (self.negpos + size) // NEGATIVE_TABLE_SIZE >= 1:
            print('reshuffle negative table...')
            np.random.shuffle(self.negatives)
        self.negpos = (self.negpos + size) % len(self.negatives)
        if len(response) != size:
            return np.concatenate((response, self.negatives[0:self.negpos]))
        return response

    def initTableNegatives(self, ns_exponent):
        print('Initializing negative samples', flush=True)
        page_id, page_counts = zip(*self.page_frequency.items())
        ratio = np.array(page_counts).astype(np.float64) ** ns_exponent / sum(page_counts)
        sampled_count = np.round(ratio * NEGATIVE_TABLE_SIZE)

        df = pd.DataFrame.from_records(enumerate(sampled_count))
        # the column 0 is the page id, column 1 is the count of the page
        self.negatives = np.repeat(df[0].astype(np.int64).values, df[1].astype(np.int64).values)
        np.random.shuffle(self.negatives)

    def collate(self,batches):
        negs = self.getNegatives(None, self.num_negs * len(batches)).reshape((len(batches), self.num_negs))
        id_list, positive_list = zip(*batches)

        return torch.LongTensor(id_list), torch.LongTensor(positive_list), torch.from_numpy(negs)

    @staticmethod
    def worker_init_fn(worker_id, file_handle_lists):
        worker_info = torch.utils.data.get_worker_info()
        dataset = worker_info.dataset  # the dataset copy in this worker process
        worker_id = worker_info.id
        dataset.file_list = file_handle_lists[worker_id]

In [8]:
#%pdb

In [9]:
def test():
    tt = get_file_handles_in_zip(LINK_PAIRS_LOCATION)
    for i, d in enumerate(WikiDataset(tt, compression='zip', n_chunk=100, num_negs=5, page_word_stats = page_word_stats, ns_exponent = 0.5)):
        print(d)
        if i > 100:
            break


In [10]:
#%lprun -f WikiDataset.__next__ test()

In [11]:
# # Datasets
# partition = # IDs
# labels = # Labels

# #dataloader -> iterator
# training_set = Dataset(partition['train'], labels)
# validation_set = Dataset(partition['validation'], labels)

# #TODO: load trainset and testset
# training_generator  = DataLoader(trainset, **data_params)
# validation_generator  = DataLoader(testset, **data_params)

Model

In [12]:
class two_tower(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, hidden_dim1, out_dim):
        super(nn.Module, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden_dim1)
        self.linear2 = nn.Linear(hidden_dim1, out_dim)

    def forward_left(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        h = F.relu(self.linear1(embeds))
        out = F.relu(self.linear2(h))
        # log_probs = F.log_softmax(out, dim=1)
        return out

    def forward_right(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        h = F.relu(self.linear1(embeds))
        out = F.relu(self.linear2(h))
        # log_probs = F.log_softmax(out, dim=1)
        return out

    def forward(self, inputs):
        inputs_left, inputs_right = inputs
        return self.forward_left(inputs_left), self.forward_right(inputs_right)

In [13]:
class OneTower(nn.Module):
    def __init__(self, corpus_size, embedding_dim, hidden_dim1, out_dim, sparse, single_layer = False):
        super(OneTower, self).__init__()
        self.embeddings = nn.Embedding(corpus_size, embedding_dim, sparse=sparse)
        self.single_layer = single_layer

        # if single_layer is True, it essentially become w2v model with single hidden layer
        if not self.single_layer:
            self.linear1 = nn.Linear(embedding_dim, hidden_dim1)
            self.linear2 = nn.Linear(hidden_dim1, out_dim)
        self.out_embeddings = nn.Embedding(corpus_size, embedding_dim, sparse=sparse)
        
    def forward(self, pos_u, pos_v, neg_v):
        emb_u = self.embeddings(pos_u)
        emb_v = self.out_embeddings(pos_v)
        emb_neg_v = self.out_embeddings(neg_v)

        if not self.single_layer:
            h1 = F.relu(self.linear1(emb_u))
            h2 = F.relu(self.linear2(h1))
        else:
            h2 = emb_v

        score = torch.sum(torch.mul(h2, emb_v), dim=1)
        score = torch.clamp(score, max=10, min=-10)
        score = -F.logsigmoid(score)


        neg_score = torch.bmm(emb_neg_v, h2.unsqueeze(2)).squeeze()
        neg_score = torch.clamp(neg_score, max=10, min=-10)
        neg_score = -torch.sum(F.logsigmoid(-neg_score), dim=1)

        return torch.mean(score + neg_score)    

In [14]:
%pdb

Automatic pdb calling has been turned ON


In [15]:
class MultipleOptimizer:
    def __init__(self, *op):
        self.optimizers = op

    def zero_grad(self):
        for op in self.optimizers:
            op.zero_grad()

    def step(self):
        for op in self.optimizers:
            op.step()

class WikiTrainer:

    def __init__(self, page_word_stats, hidden_dim1, out_dim, embedding_dim=100, batch_size=32, window_size=5, iterations=3,
                 initial_lr=0.001, min_count=12, num_workers=0, collate_fn='custom', iprint=500, t=1e-3, ns_exponent=0.75, 
                 optimizer='adam', optimizer_kwargs = None, warm_start_model = None, lr_schedule = False, timeout = 60, n_chunk = 20,
                 sparse=False, single_layer=False, test = False, save_embedding = True):

        file_handle_lists = get_files_in_dir(LINK_PAIRS_LOCATION)
        if test:
            file_handle_lists = file_handle_lists[:2]
            num_workers = 0
            n_chunk = 1
            
        dataset = WikiDataset(file_list = None, compression = 'zip', n_chunk = n_chunk, 
                              page_word_stats = page_word_stats, num_negs=5, 
                              ns_exponent=ns_exponent)
        if collate_fn == 'custom':
            collate_fn = dataset.collate
        else:
            collate_fn = None
        
        if num_workers > 0:
            file_handle_lists = np.array_split(file_handle_lists, num_workers)
        else:
            timeout = 0
            dataset.file_list = file_handle_lists

        self.dataloader = DataLoader(dataset, batch_size=batch_size,
                                     shuffle=False, num_workers=num_workers, 
                                     collate_fn=collate_fn, 
                                     worker_init_fn=partial(dataset.worker_init_fn, file_handle_lists=file_handle_lists),
                                     timeout = timeout,
                                    )

        # self.output_file_name = output_file
        self.corpus_size = len(dataset.page_frequency)
        self.embedding_dim = embedding_dim
        self.save_embedding = save_embedding
        self.iprint = iprint
        self.batch_size = batch_size
        self.iterations = iterations
        self.initial_lr = initial_lr
        self.model = OneTower(self.corpus_size, self.embedding_dim, 
 #                             context_size = dataset.num_neg, 
                              hidden_dim1 = hidden_dim1, 
                              out_dim = out_dim, sparse=sparse,
                              single_layer = single_layer,
                              )

        if warm_start_model is not None:
            self.model.load_state_dict(torch.load(warm_start_model), strict=False)
        self.optimizer = optimizer
        if optimizer_kwargs is None:
            optimizer_kwargs = {}
        self.optimizer_kwargs = optimizer_kwargs
        self.lr_schedule = lr_schedule
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")
        if self.use_cuda:
            self.model.cuda()

    def train(self):
        # clearn GPU memory cache
        gc.collect()
        torch.cuda.empty_cache()

        if self.optimizer == 'adam':
            optimizer = optim.Adam(self.model.parameters(), lr=self.initial_lr, **self.optimizer_kwargs)
        elif self.optimizer == 'sparse_adam':
            optimizer = optim.SparseAdam(self.model.parameters(), lr=self.initial_lr, **self.optimizer_kwargs)
        elif self.optimizer == 'sparse_dense_adam':
            opti_sparse = optim.SparseAdam([self.model.embeddings.weight, self.model.out_embeddings.weight], lr=self.initial_lr, **self.optimizer_kwargs)
            opti_dense = optim.Adam([self.model.linear1.weight, self.model.linear2.weight], lr=self.initial_lr, **self.optimizer_kwargs)
            optimizer = MultipleOptimizer(opti_sparse, opti_dense)
        elif self.optimizer == 'sgd':
            optimizer = optim.SGD(self.model.parameters(), lr=self.initial_lr, **self.optimizer_kwargs)
        elif self.optimizer == 'asgd':
            optimizer = optim.ASGD(self.model.parameters(), lr=self.initial_lr, **self.optimizer_kwargs)
        elif self.optimizer == 'adagrad':
            optimizer = optim.Adagrad(self.model.parameters(), lr=self.initial_lr, **self.optimizer_kwargs)
        else:
            raise Exception('Unknown optimizer!')

        for iteration in range(self.iterations):

            print("\n\n\nIteration: " + str(iteration + 1))

            if self.lr_schedule:
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(self.dataloader))
            running_loss = 0.0
            iprint = self.iprint #len(self.dataloader) // 20
            for i, sample_batched in enumerate(self.dataloader):
                # ipdb.set_trace()
                if len(sample_batched[0]) > 1:
                    pos_u = sample_batched[0].to(self.device)
                    pos_v = sample_batched[1].to(self.device)
                    neg_v = sample_batched[2].to(self.device)
                    
                    optimizer.zero_grad()
                    loss = self.model.forward(pos_u, pos_v, neg_v)
                    loss.backward()
                    optimizer.step()
                    if self.lr_schedule:
                        scheduler.step()

                    running_loss = running_loss * (1 - 5/iprint) + loss.item() * (5/iprint)
                    if i > 0 and i % iprint == 0:
                        # print(" Loss: " + str(running_loss) + ' lr: ' 
                        #     + str([param_group['lr'] for param_group in optimizer.param_groups]))
                        print(" Loss: " + str(running_loss))
            print(" Loss: " + str(running_loss))

        #self.skip_gram_model.save_embedding(self.data.id2word, self.output_file_name)
        if self.save_embedding:
            print('Saving embeddings...', flush=True)
            embeddings = self.model.embeddings.weight.cpu().data.numpy()
            np.savez_compressed('gdrive/My Drive/Projects with Wei/wiki_data/wiki_embedding/embedding.npz', embeddings)

In [16]:
def train_model(epochs = 1, collate_fn='custom',num_workers=1, test = False, n_chunk = 20, save_embedding=True):
  #ipdb.set_trace()
  wt = WikiTrainer(
                        page_word_stats = page_word_stats, 
                        hidden_dim1 = 100, 
                        out_dim = 100, 
                        # output_file="gdrive/My Drive/Projects with Wei/Wei_tmp_outputs/w2v_output/out.vec",
                        # min_count=5,
                        batch_size=8096,
                        iterations=epochs,
                        num_workers=num_workers,
                        collate_fn=collate_fn,
                        iprint=5000,
                        n_chunk = n_chunk,
                        embedding_dim=100,
                        ns_exponent=0.75,
                        initial_lr=0.003,
                        optimizer='sparse_adam',
                        single_layer=True,
                        sparse=True,
                        lr_schedule=False,
                        test=test,
                        save_embedding=save_embedding,
 #                       optimizer_kwargs={'momentum':0.9},
 #                       warm_start_model = 'gdrive/My Drive/Projects with Wei/Wei_tmp_outputs/torch_model/model.pkl',
                        )
  wt.train()
  return wt

In [17]:
gc.collect()
torch.cuda.empty_cache()

In [18]:
train_model(epochs=10,
            num_workers=3, 
            test = False, 
            save_embedding=True)

Initializing negative samples



Iteration: 1


  5%|▌         | 1/20 [01:07<21:23, 67.56s/it]

 Loss: 14.323855716760006


 15%|█▌        | 3/20 [03:20<18:58, 66.98s/it]

 Loss: 13.435222632375664


...

 Loss: 3.3524024918375086


 95%|█████████▌| 19/20 [18:56<00:56, 56.16s/it]

 Loss: 3.349394866896453


100%|██████████| 20/20 [19:46<00:00, 59.33s/it]


 Loss: 3.3468876963998517
Saving embeddings...


Train

In [ ]:
%lprun -f WikiDataset.__next__ -f WikiDataset.collate -f WikiTrainer.train train_model(num_workers=0, test = True, n_chunk=1)

Initializing negative samples



  0%|          | 0/1 [00:00<?, ?it/s]




Iteration: 1



100%|██████████| 1/1 [00:58<00:00, 58.34s/it]


 Loss: 5.873366421972997


In [ ]:
length = 0
for c in read_link_pairs_chunks(n_chunk=10):
    print(len(c))
    length += len(c)
print(length)

  0%|          | 0/10 [00:00<?, ?it/s]

reading link pairs in 10 chunks


 10%|█         | 1/10 [00:11<01:46, 11.87s/it]

37397160


 20%|██        | 2/10 [00:22<01:32, 11.62s/it]

...

37397160


 90%|█████████ | 9/10 [01:37<00:10, 10.56s/it]

37397160


100%|██████████| 10/10 [01:48<00:00, 10.80s/it]

37397113
373971553


In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, outputs, label):
        outputs1, output2 = outputs
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

We'll define a little function to create our model and optimizer so we
can reuse it in the future.

In [ ]:
def get_model(C, device):
    model = two_tower().to(device)
    return model, optim.Adam(model.parameters(), lr = C.LEARNING_RATE)

Since we go through a similar process twice of calculating the loss for both the training set and the validation set, let's make that into its own function, loss_batch, which computes the loss for one batch.

We pass an optimizer in for the training set, and use it to perform backprop. For the validation set, we don't pass an optimizer, so the method doesn't perform backprop.

In [ ]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

fit runs the necessary operations to train our model and compute the training and validation losses for each epoch. 

(Note that we always call model.train() before training, and model.eval() before inference, because these are used by layers such as nn.BatchNorm2d and nn.Dropout to ensure appropriate behaviour for these different phases.)



In [ ]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl, device):
    counter = []
    loss_history = [] 
    iteration_number= 0
    
    for epoch in range(epochs):
        model.train()
        for i, data in enumerate(train_dl, 0):
            xb, yb = data
            loss_batch(model, loss_func, xb, yb, opt)
            
            if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch, loss_contrastive.data[0]))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.data[0])

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        print(epoch, val_loss)

    show_plot(counter,loss_history)

"get_data" returns dataloaders for the training and validation sets.

In [ ]:
#TODO: send the batches to device

def get_data(X, Y, C):
    # Datasets
    X = # IDs
    Y = # Labels

    #dataloader -> iterator
    training_set = Dataset(X['train'], Y)
    validation_set = Dataset(X['validation'], Y)

    return (
        DataLoader(train_ds, **C.data_params),
        DataLoader(valid_ds, **C.data_params),
    )

main(). 

Now, our whole process of obtaining the data loaders and fitting the model can be run in 3 lines of code:

In [ ]:
train_dl, valid_dl = get_data(X, Y, Config)
model, opt = get_model(Config, device)
fit(Config.NUM_EPOCHS, model, loss_func, opt, train_dl, valid_dl)

In [ ]:
#@title
model, opt = get_model()
loss_func  = ContrastiveLoss()
print(loss(model(xb), yb))


counter = []
loss_history = [] 
iteration_number= 0

for epoch in range(0, Config.NUM_EPOCHS):
    model.train()
    for i, xb1, xb2 in enumerate(train_dataloader, 0):
        pred1, pred2 = model(xb1, xb2)
        loss_contrastive = loss_func(pred1, pred2, yb)
        loss_contrastive.backward()
        opt.step()
        opt.zero_grad()
  
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch, loss_contrastive.data[0]))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.data[0])
    
    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb1, xb2), yb) for xb1, xb2, yb in validation_generator)
    print(epoch, valid_loss / len(validation_generator))     
       
show_plot(counter,loss_history)